In [4]:
import pandas as pd

file_path = "Walmart_Sales.csv"
df = pd.read_csv(file_path)
print(df.iloc[:100].to_csv())


questions = [
    "Apakah penjualan mingguan meningkat secara signifikan ketika ada libur besar dan promosi dilakukan bersamaan?",
    "Bagaimana pengaruh suhu dan harga bahan bakar terhadap penjualan mingguan di tiap region?",
    "Apakah toko berukuran besar lebih tahan terhadap dampak inflasi (CPI) dan pengangguran dibanding toko kecil?",
    "Apakah dampak libur besar pada penjualan konsisten di semua toko dan semua region, atau ada variasi?",
    "Apakah promosi tetap efektif meningkatkan penjualan meskipun harga bahan bakar tinggi dan inflasi naik?",
    "Bagaimana tren penjualan mingguan selama setahun terakhir di toko besar vs kecil, terutama pada minggu libur?"
]

print("mohon jawab pertanyaan berikut")
for i in questions:
    print(i)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.9,0,42.31,2.572,211.0963582,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.2421698,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.2891429,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.3196429,8.106
4,1,05-03-2010,1554806.68,0,46.5,2.625,211.3501429,8.106
5,1,12-03-2010,1439541.59,0,57.79,2.667,211.3806429,8.106
6,1,19-03-2010,1472515.79,0,54.58,2.72,211.215635,8.106
7,1,26-03-2010,1404429.92,0,51.45,2.732,211.0180424,8.106
8,1,02-04-2010,1594968.28,0,62.27,2.719,210.8204499,7.808
9,1,09-04-2010,1545418.53,0,65.86,2.77,210.6228574,7.808
10,1,16-04-2010,1466058.28,0,66.32,2.808,210.4887,7.808
11,1,23-04-2010,1391256.12,0,64.84,2.795,210.4391228,7.808
12,1,30-04-2010,1425100.71,0,67.41,2.78,210.3895456,7.808
13,1,07-05-2010,1603955.12,0,72.55,2.835,210.3399684,7.808
14,1,14-05-2010,1494251.5,0,74.78,2.854,210.3374261,7.808
15,1,21-05-2010,1399662.07,0,76.44,2.826,210.6170934,7.80

In [5]:
import pandas as pd
import numpy as np

# === 1. Baca dataset ===
file_path = "Walmart_Sales.csv"   # ganti sesuai dataset kamu
df = pd.read_csv(file_path)
df = df.iloc[:100]

# Ubah Date ke datetime
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

questions = [
    "Apakah penjualan mingguan meningkat secara signifikan ketika ada libur besar dan promosi dilakukan bersamaan?",
    "Bagaimana pengaruh suhu dan harga bahan bakar terhadap penjualan mingguan di tiap region?",
    "Apakah toko berukuran besar lebih tahan terhadap dampak inflasi (CPI) dan pengangguran dibanding toko kecil?",
    "Apakah dampak libur besar pada penjualan konsisten di semua toko dan semua region, atau ada variasi?",
    "Apakah promosi tetap efektif meningkatkan penjualan meskipun harga bahan bakar tinggi dan inflasi naik?",
    "Bagaimana tren penjualan mingguan selama setahun terakhir di toko besar vs kecil, terutama pada minggu libur?"
]

# === 2. Analisis sederhana sebagai kunci jawaban ===
keys = {}

# Buat pembagian toko besar/kecil berdasarkan median sales
sales_per_store = df.groupby("Store")["Weekly_Sales"].mean()
median_sales = sales_per_store.median()
big_stores = sales_per_store[sales_per_store >= median_sales].index
small_stores = sales_per_store[sales_per_store < median_sales].index

# Q1: libur besar + promosi (anggap promosi = Holiday_Flag)
if "Holiday_Flag" in df.columns:
    mean_holiday = df[df["Holiday_Flag"] == 1]["Weekly_Sales"].mean()
    mean_nonholiday = df[df["Holiday_Flag"] == 0]["Weekly_Sales"].mean()
    keys[questions[0]] = f"Rata-rata penjualan libur besar: {mean_holiday:.2f}, non-libur: {mean_nonholiday:.2f}. {'Naik' if mean_holiday>mean_nonholiday else 'Turun'}"
else:
    keys[questions[0]] = "Data Holiday_Flag tidak tersedia."

# Q2: suhu & harga bahan bakar vs penjualan
if "Temperature" in df.columns and "Fuel_Price" in df.columns:
    corr_temp = df["Weekly_Sales"].corr(df["Temperature"])
    corr_fuel = df["Weekly_Sales"].corr(df["Fuel_Price"])
    keys[questions[1]] = f"Korelasi suhu vs penjualan: {corr_temp:.2f}, fuel vs penjualan: {corr_fuel:.2f}"
else:
    keys[questions[1]] = "Data Temperature/Fuel_Price tidak tersedia."

# Q3: stabilitas terhadap CPI & Unemployment
if "CPI" in df.columns and "Unemployment" in df.columns:
    big_var = df[df["Store"].isin(big_stores)][["CPI","Unemployment","Weekly_Sales"]].corr().loc[["CPI","Unemployment"],"Weekly_Sales"]
    small_var = df[df["Store"].isin(small_stores)][["CPI","Unemployment","Weekly_Sales"]].corr().loc[["CPI","Unemployment"],"Weekly_Sales"]
    keys[questions[2]] = f"Korelasi toko besar: {big_var.to_dict()}, toko kecil: {small_var.to_dict()}"
else:
    keys[questions[2]] = "Data CPI/Unemployment tidak tersedia."

# Q4: variasi libur antar toko
if "Holiday_Flag" in df.columns and "Store" in df.columns:
    holiday_effects = df.groupby(["Store","Holiday_Flag"])["Weekly_Sales"].mean().unstack()
    variation = holiday_effects.std(axis=1).mean()
    keys[questions[3]] = f"Variasi rata-rata antar toko: {variation:.2f}"
else:
    keys[questions[3]] = "Kolom Store/Holiday_Flag tidak tersedia."

# Q5: promosi vs kondisi ekonomi
if all(col in df.columns for col in ["Fuel_Price","CPI"]):
    corr_fuel = df["Weekly_Sales"].corr(df["Fuel_Price"])
    corr_cpi = df["Weekly_Sales"].corr(df["CPI"])
    keys[questions[4]] = f"Korelasi fuel: {corr_fuel:.2f}, CPI: {corr_cpi:.2f}"
else:
    keys[questions[4]] = "Data Fuel_Price/CPI tidak tersedia."

# Q6: tren setahun terakhir
if not df["Date"].isna().all():
    last_year = df[df["Date"] >= df["Date"].max() - pd.DateOffset(years=1)]
    trend_big = last_year[last_year["Store"].isin(big_stores)]["Weekly_Sales"].mean()
    trend_small = last_year[last_year["Store"].isin(small_stores)]["Weekly_Sales"].mean()
    keys[questions[5]] = f"Rata-rata setahun terakhir - Toko besar: {trend_big:.2f}, Toko kecil: {trend_small:.2f}"
else:
    keys[questions[5]] = "Kolom Date tidak bisa diproses."

# === 3. Output pertanyaan + kunci jawaban ===
print("Mohon jawab pertanyaan berikut. Setelah itu akan dibandingkan dengan kunci jawaban.")
for q in questions:
    print(f"\nPertanyaan: {q}")
    print(f"Kunci Jawaban (berdasarkan data): {keys[q]}")
    print("Skor: 0-10 (tentukan berdasarkan ketepatan jawaban Anda)")


Mohon jawab pertanyaan berikut. Setelah itu akan dibandingkan dengan kunci jawaban.

Pertanyaan: Apakah penjualan mingguan meningkat secara signifikan ketika ada libur besar dan promosi dilakukan bersamaan?
Kunci Jawaban (berdasarkan data): Rata-rata penjualan libur besar: 1649153.98, non-libur: 1532690.42. Naik
Skor: 0-10 (tentukan berdasarkan ketepatan jawaban Anda)

Pertanyaan: Bagaimana pengaruh suhu dan harga bahan bakar terhadap penjualan mingguan di tiap region?
Kunci Jawaban (berdasarkan data): Korelasi suhu vs penjualan: -0.28, fuel vs penjualan: 0.05
Skor: 0-10 (tentukan berdasarkan ketepatan jawaban Anda)

Pertanyaan: Apakah toko berukuran besar lebih tahan terhadap dampak inflasi (CPI) dan pengangguran dibanding toko kecil?
Kunci Jawaban (berdasarkan data): Korelasi toko besar: {'CPI': 0.2583458888110231, 'Unemployment': 0.004017594378400096}, toko kecil: {'CPI': nan, 'Unemployment': nan}
Skor: 0-10 (tentukan berdasarkan ketepatan jawaban Anda)

Pertanyaan: Apakah dampak li

In [8]:
print(df.columns)


Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')


In [9]:
print(df.head())

   Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1 2010-02-05    1643690.90             0        42.31       2.572   
1      1 2010-02-12    1641957.44             1        38.51       2.548   
2      1 2010-02-19    1611968.17             0        39.93       2.514   
3      1 2010-02-26    1409727.59             0        46.63       2.561   
4      1 2010-03-05    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  
